## Figures & Data for OPTIMIST Presentation

In [ ]:
from matplotlib import pyplot as plt
from time import perf_counter_ns
import os
import numpy as np

from src.zarr_handler import ZarrHandler
from src.generic_operation import OracleAES, SNR, NICV

def presentation_plot(data: np.ndarray, plot_size: tuple[int, int], lq_name: str, subtitle: str, yscale: tuple[float|int, float|int]):
    plt.figure(figsize=plot_size)
    for i in range(data.shape[0]):
        plt.plot(np.arange(data.shape[1]), data[i, :], label=f"byte {i}", color=(i/data.shape[0], 0, 1-(i/data.shape[0])))
        ax = plt.gca()
        ax.set_ylim(bottom=yscale[0], top=yscale[1])
        plt.legend(loc='upper right')
        plt.title(f"{lq_name}\n{subtitle}")
        plt.xlabel("Time sample")
        plt.ylabel(f"{lq_name} (leakage)")
    plt.savefig(f"figure-{subtitle.lower()}.png", dpi=300)
    # plt.show()

# import dataset
def presentation_dataset(name: str) -> str:
    return os.path.join("data", name + '.zarr')
dataset_name = "SAM4S 100000 random Traces [50c]"
dset = ZarrHandler(presentation_dataset(dataset_name), chunks=10000)

# get data
k = dset.get("key")
ct = dset.get("ciphertext")
t = dset.get("traces")

# we use SNR for leakage quantification
snr = SNR(t, ct.compute())
presentation_plot(snr.snr, (10, 10), "SNR", f"Target: Ciphertext (input)", (0, 1.6))

# calculate leakage for each step in the first round of AES
for target in ((1, 'sbox'), (1, 'shift'), (1, 'mix'), (1, 'add')):
    # instantiate oracle
    oracle = OracleAES(
        aes_type=128, keys=k, texts=ct, 
        round=target[0], step=target[1],    # finds intermediate state at round 'round' step 'step'
        lazy=False                          # lazy=False computes results immedietly on init
    )

    # instantiate SNR operation
    snr = SNR(t, oracle.result)

    # plot results
    presentation_plot(snr.snr, (10, 10), "SNR", f"Target: {target[1].upper()} Round {target[0]} result", (0, 1.6))



## Proof of Concept: Comparative Analysis of Multiple Datasets

In [ ]:
dataset_names = [
    "SAM4S 100000 random Traces [50c]",
    "SAM4S 100000 random Traces [60c]"
]

datasets = [ ZarrHandler(presentation_dataset(name), chunks=10000) for name in dataset_names ]
oracles = [ OracleAES(aes_type=128, keys=dset.get("key"), texts=dset.get("ciphertext"), round=1, step='sbox', lazy=False) for dset in datasets ]
results = [ SNR(dset.get("traces"), oracle.result).snr for dset, oracle in zip(datasets, oracles) ]

for (result, name) in zip(results, dataset_names):
    presentation_plot(result, (10, 10), "SNR", f"Target: Round 1 SBOX | {name}", (0, 1.6))

presentation_plot(np.abs(np.subtract(results[0], results[1])), (10, 10), "SNR Difference", f"Target: Round 1 SBOX", (0, 0.1))
